In [ ]:
import pandas as pd

In [ ]:
jan_df = pd.read_csv('data/Jan Sold With.csv')
nov_df = pd.read_csv('data/Nov Sold With.csv')
dec_df = pd.read_csv('data/Dec Sold With.csv')
stores_df = pd.read_csv('data/stores.csv')
store_sales_df = pd.read_csv('data/Store Sales.csv')
products_df = pd.read_csv('data/products.csv')

In [ ]:
print(jan_df.shape)
jan_df.head()

In [ ]:
print(nov_df.shape)
nov_df.head()

In [ ]:
print(dec_df.info())
print(dec_df.shape)
dec_df.head()

In [ ]:
dec_df.TRANSACTION_DATE.value_counts()

In [ ]:
print(stores_df.shape)
stores_df.head()

In [ ]:
stores_df.STORE_NAME.unique()

In [ ]:
print(store_sales_df.shape)
store_sales_df[store_sales_df['STORE'] == 1500]

In [ ]:
products_df.head()

In [ ]:
products_df.shape

In [ ]:
products_df.DEPARTMENT_NAME.unique()

In [ ]:
products_df.MAJOR_NAME.unique()

In [ ]:
products_df.STYLE_DESCR.value_counts()

In [ ]:
products_df[products_df['DEPARTMENT'] == 1]

# Analysis

In [ ]:
jan_df[['PRIMARY_STOCKNO','SECONDARY_STOCKNO']].value_counts()

In [ ]:
products_df[(products_df['STOCKNO'] == 398913) | (products_df['STOCKNO'] == 1639)]

I am trying to create a function that when given a stock number it will return the top 3 shoes and top 3 accessories sold with that stock number. Once I have this I can run this on the products table to create 6 new columns. 3 for the 1st, 2nd, and 3rd shoe recomendation and 3 for the 1st, 2nd, and 3rd acessory recomendation.
<br><br>
I could add to more inputs such as age(adult/kid) and sex(male/female) which would act as filters.

#### Doing some test to see how I can write the function

In [ ]:
jan_df[(jan_df == 398913).any(axis=1)]

Given a stock number, return a dataframe with all combinations of that stock number and how many times those combinations were sold

In [ ]:
recomendation_df = pd.DataFrame(jan_df[(jan_df['PRIMARY_STOCKNO'] == 398913)][['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
recomendation_df

In [ ]:
recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']

In [ ]:
recomendation_df

Return the top 3 items that are most commonly bought with that stock number

In [ ]:
STOCK = 398913

for index, row in recomendation_df.head(10).iterrows():
    if row.SECONDARY_STOCKNO == STOCK:
        print(row.PRIMARY_STOCKNO)
    else:
        print(row.SECONDARY_STOCKNO)
        


In [ ]:
#looking into the returned stock numbers
products_df[(products_df['STOCKNO'] == 1639) | (products_df['STOCKNO'] == 398564) | (products_df['STOCKNO'] == 401)]

I am now joining the DEPARTMENT from the products table to the January sales table so I can filter on accessory or shoe. I will be joining on STOCKNO and PRIMARY_STOCKNO.

In [ ]:
jan_sales_department = pd.merge(jan_df, products_df[['STOCKNO','DEPARTMENT']], 
                               left_on = 'PRIMARY_STOCKNO', right_on = 'STOCKNO', 
                               how = 'left')
jan_sales_department.head(3)

In [ ]:
jan_sales_department = jan_sales_department.rename(columns={"DEPARTMENT": "PRIMARY_DEPT"})

In [ ]:
jan_sales_department.head(3)

Doing the same thing but for SECONDARY_STOCKNO

In [ ]:
jan_sales_departments = pd.merge(jan_sales_department, products_df[['STOCKNO','DEPARTMENT']], 
                               left_on = 'SECONDARY_STOCKNO', right_on = 'STOCKNO', 
                               how = 'left')
jan_sales_departments.head(3)

Dropping some columns

In [ ]:
jan_sales_departments = jan_sales_departments.drop(columns = ['TRANSACTION_DATE', 'PRIMARY_EMP_IND',
                                                              'SECONDARY_EMP_IND', 'PRIMARY_SOS_IND',
                                                              'SECONDARY_SOS_IND', 'PRIMARY_LINENO',
                                                              'SECONDARY_LINENO', 'STOCKNO_x',
                                                              'STOCKNO_y' ])

In [ ]:
jan_sales_departments = jan_sales_departments.rename(columns={"DEPARTMENT": "SECONDARY_DEPT"})

In [ ]:
jan_sales_departments.head(5)

Creating a new column PRIMARY_SHOE (True or False)

In [ ]:

for index, row in jan_sales_departments.iterrows():
    if row.PRIMARY_DEPT in [1, 2, 3, 4, 7]:
        jan_sales_departments.loc[index, 'PRIMARY_SHOE'] = 'True'
    elif row.PRIMARY_DEPT in [5, 6, 8, 9, 10, 15]:
        jan_sales_departments.loc[index, 'PRIMARY_SHOE'] = 'False'
    else:
        jan_sales_departments.loc[index, 'PRIMARY_SHOE'] = 'error'

In [ ]:
jan_sales_departments.head(3)

Creating a new column SECONDARY_SHOE (True or False)

In [ ]:
for index, row in jan_sales_departments.iterrows():
    if row.SECONDARY_DEPT in [1, 2, 3, 4, 7]:
        jan_sales_departments.loc[index, 'SECONDARY_SHOE'] = 'True'
    elif row.SECONDARY_DEPT in [5, 6, 8, 9, 10, 15]:
        jan_sales_departments.loc[index, 'SECONDARY_SHOE'] = 'False'
    else:
        jan_sales_departments.loc[index, 'SECONDARY_SHOE'] = 'error'

In [ ]:
jan_sales_departments.head(3)

Creating a table for shoes sold with shoes (jan_shoe_shoe), and shoes sold with accessories (jan_shoe_acces)

In [ ]:
jan_shoe_shoe = jan_sales_departments[(jan_sales_departments['SECONDARY_SHOE'] == 'True') 
                                      & (jan_sales_departments['PRIMARY_SHOE'] == 'True')]

jan_shoe_acces = jan_sales_departments[(jan_sales_departments['SECONDARY_SHOE'] == 'False')
                                      & (jan_sales_departments['PRIMARY_SHOE'] == 'True')]

### The following functions are used to find the top 3 shoe or accessory recomendations given a shoe's stock number. The first list contains the recomendations. The second list returns the amount of times this combonation has been purchased.

In [ ]:
def shoe_recomendations(stock_num):
    
    recomendation_df = pd.DataFrame(jan_shoe_shoe[(jan_shoe_shoe['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count

In [ ]:
shoe_recomendations(398913)

In [ ]:
def acces_recomendation(stock_num):
    
    recomendation_df = pd.DataFrame(jan_shoe_acces[(jan_shoe_acces['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count

In [ ]:
acces_recomendation(398913)

### I want to be able to filter on location hierarchy so I am joining the stores table to the sales table using 'store'. I am also merging all the sales tables (nov. ,dec., and jan.) into one df called sales_df.

In [ ]:
sales_df = pd.concat([nov_df, dec_df, jan_df])
sales_df.head(3)